In [ ]:
# 비트코인, 이더리움, FTX Token의 가격 및 거래량의 일별 자료를 통해 비트코인 과의 상관관계를 알아보고자 함.
# 3개년 치 (2020-10-26 ~ 2022-12-26) 의 자료를 investing.com 의 과거 데이터 다운로드를 통해 자료 수집.
# 데이터 전처리 후 matplotlib를 통해 각종 도표로 데이터 확인.

In [285]:
import pandas as pd

In [286]:
# %load matplot.py
# %%
import matplotlib.pyplot as plt
plt.rc('font', family = 'gulim')
plt.rc('axes', unicode_minus=False)




In [287]:
bit_origin = pd.read_csv('./data/bit.csv')
eth_origin = pd.read_csv('./data/eth.csv')
wemix_origin = pd.read_csv('./data/wemix.csv')

# 각 데이터 프레임을 copy에 할당함
bit_copy, eth_copy, ftt_copy = bit_origin.copy(), eth_origin.copy(), wemix_origin.copy()
print(bit_copy.head())

              날짜        종가        오픈        고가        저가      거래량    변동 %
0  2022년 12월 26일  16,831.9  16,831.8  16,836.5  16,822.0  133.33K   0.00%
1  2022년 12월 25일  16,831.8  16,835.7  16,852.9  16,733.0  132.89K  -0.03%
2  2022년 12월 24일  16,837.2  16,778.6  16,855.8  16,777.8  110.12K   0.35%
3  2022년 12월 23일  16,779.1  16,820.5  16,910.7  16,768.0  184.12K  -0.25%
4  2022년 12월 22일  16,820.6  16,831.8  16,862.2  16,566.1  198.28K  -0.07%


### bit_copy 데이터 전처리

In [288]:
bit_copy.isnull().sum() # 다행히 NaN 값은 없음.
bit_copy.info() # 모든 컬럼의 데이터 타입이 object로 되어있음.
# 종가, 오픈, 고가, 저가의 컬럼 데이터 타입을 float로 변경하고자 함.
bit_copy['종가'] = bit_copy['종가'].astype('float')
# string 데이터에 comma(,)가 있기 때문에 바로 전환이 안됨.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      761 non-null    object
 1   종가      761 non-null    object
 2   오픈      761 non-null    object
 3   고가      761 non-null    object
 4   저가      761 non-null    object
 5   거래량     761 non-null    object
 6   변동 %    761 non-null    object
dtypes: object(7)
memory usage: 41.7+ KB


ValueError: could not convert string to float: '16,831.9'

In [289]:
# Python: Convert string with comma separator and dot to float
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

bit_copy['종가'] = bit_copy['종가'].map(locale.atof)

In [290]:
bit_copy['종가'].dtype # '종가'컬럼의 데이터 타입이 float64로 변경됨

dtype('float64')

In [291]:
# bit_copy[['오픈','고가','저가']] = bit_copy[['오픈','고가','저가']].map(locale.atof)
# 위처럼 하니 오류가 발생하여 그냥 하나씩 진행함
bit_copy['오픈'] = bit_copy['오픈'].map(locale.atof)
bit_copy['고가'] = bit_copy['고가'].map(locale.atof)
bit_copy['저가'] = bit_copy['저가'].map(locale.atof)

In [292]:
bit_copy.info() # 오픈, 고가, 저가의 데이터 타입이 float64로 변경된 것을 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      761 non-null    object 
 1   종가      761 non-null    float64
 2   오픈      761 non-null    float64
 3   고가      761 non-null    float64
 4   저가      761 non-null    float64
 5   거래량     761 non-null    object 
 6   변동 %    761 non-null    object 
dtypes: float64(4), object(3)
memory usage: 41.7+ KB


In [293]:
bit_copy['거래량'].head() # 거래량의 K는 1000을 뜻함. 이를 K를 없애고 일반 숫자로 변환하고 싶음.

0    133.33K
1    132.89K
2    110.12K
3    184.12K
4    198.28K
Name: 거래량, dtype: object

In [294]:
for i in range(len(bit_copy['거래량'])):
    bit_copy['거래량'][i] = bit_copy['거래량'][i][:-1] # 마지막 단어 K를 없앰
bit_copy['거래량'].head()

C:\Users\Jasper\AppData\Local\Temp\ipykernel_26092\1011294417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_copy['거래량'][i] = bit_copy['거래량'][i][:-1] # 마지막 단어 K를 없앰


0    133.33
1    132.89
2    110.12
3    184.12
4    198.28
Name: 거래량, dtype: object

In [295]:
bit_copy['거래량'] = bit_copy['거래량'].map(locale.atof) #거래량을 float64로 변경
bit_copy['거래량'] *= 1000 # 단위수 맞춤

In [296]:
bit_copy.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2022년 12월 26일,16831.9,16831.8,16836.5,16822.0,133330.0,0.00%
1,2022년 12월 25일,16831.8,16835.7,16852.9,16733.0,132890.0,-0.03%
2,2022년 12월 24일,16837.2,16778.6,16855.8,16777.8,110120.0,0.35%
3,2022년 12월 23일,16779.1,16820.5,16910.7,16768.0,184120.0,-0.25%
4,2022년 12월 22일,16820.6,16831.8,16862.2,16566.1,198280.0,-0.07%


In [297]:
# 날짜를 연, 월, 일로 분리하고 싶음
bit_copy['날짜'].head()

0    2022년 12월 26일
1    2022년 12월 25일
2    2022년 12월 24일
3    2022년 12월 23일
4    2022년 12월 22일
Name: 날짜, dtype: object

In [298]:
# bit_copy_date = bit_copy.copy()
# 날짜의 데이터는 '2022년 12월 26일'로 구분자가 3개가 있음.
# 구분자를 하나로 변경.
bit_copy['날짜'] = bit_copy['날짜'].apply(lambda x: x.replace('년 ','-').replace('월 ', '-').replace('일', ''))

dates = bit_copy['날짜'].str.split('-') # 리스트로 데이터 리턴
bit_copy['연'] = dates.str.get(0)
bit_copy['월'] = dates.str.get(1)
bit_copy['일'] = dates.str.get(2)

In [299]:
bit_copy.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %,연,월,일
0,2022-12-26,16831.9,16831.8,16836.5,16822.0,133330.0,0.00%,2022,12,26
1,2022-12-25,16831.8,16835.7,16852.9,16733.0,132890.0,-0.03%,2022,12,25
2,2022-12-24,16837.2,16778.6,16855.8,16777.8,110120.0,0.35%,2022,12,24
3,2022-12-23,16779.1,16820.5,16910.7,16768.0,184120.0,-0.25%,2022,12,23
4,2022-12-22,16820.6,16831.8,16862.2,16566.1,198280.0,-0.07%,2022,12,22


In [300]:
# '변동 %'를 '변동률'로 컬럼명 변경 -> 추후 drop 해도 될듯
bit_copy.rename(columns={'변동 %':'변동률'}, inplace=True)

In [301]:
# 변동률을 계산에 활용할 수 있게끔 float로 변경
for i in range(len(bit_copy['변동률'])):
    bit_copy['변동률'][i] = bit_copy['변동률'][i][:-1] # 마지막 단어 %를 없앰
bit_copy['변동률'] = bit_copy['변동률'].map(locale.atof) #거래량을 float64로 변경
bit_copy['변동률'] /= 100 # 퍼센트 단위수 맞춤
bit_copy['변동률'].head()

C:\Users\Jasper\AppData\Local\Temp\ipykernel_26092\418020374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_copy['변동률'][i] = bit_copy['변동률'][i][:-1] # 마지막 단어 %를 없앰


0    0.0000
1   -0.0003
2    0.0035
3   -0.0025
4   -0.0007
Name: 변동률, dtype: float64

In [302]:
# '오픈'을 '시가'로 컬럼명 변경
bit_copy.rename(columns={'오픈':'시가'}, inplace=True)

In [303]:
# '종가'를 '당일종가'로 컬럼명 변경
bit_copy.rename(columns={'종가':'당일종가'}, inplace=True)

In [304]:
# '전일종가' 컬럼을 추가, 변동률 계산 확인을 위해..?
bit_copy_last = bit_copy.copy()
bit_copy_last['전일종가'] = bit_copy_last.loc[:,'당일종가'].shift(-1)
bit_copy_last.tail() # 마지막 열의 전일 종가가 없음. 변동률을 확인하여 임의로 대입
# a = (b * c) + b
# bit_copy_last['전일종가']

,날짜,당일종가,시가,고가,저가,거래량,변동률,연,월,일,전일종가
756,2020-11-30,19698.1,18186.0,19831.2,18186.0,185590.0,0.0832,2020,11,30,18185.5
757,2020-11-29,18185.5,17729.7,18336.3,17530.4,79120.0,0.0257,2020,11,29,17730.7
758,2020-11-28,17730.7,17142.6,17874.4,16874.4,95200.0,0.0352,2020,11,28,17127.1
759,2020-11-27,17127.1,17160.9,17450.3,16481.6,138800.0,-0.0020,2020,11,27,17162.0
760,2020-11-26,17162.0,18721.9,18894.9,16235.2,312070.0,-0.0834,2020,11,26,NaN


In [305]:
# '종류'라는 컬럼에 '비트코인' 추가
bit_copy['종류'] = '비트코인'

In [306]:
# 데이터프레임 컬럼 순서 변경
# bit_copy = bit_copy[['종류','날짜','연','월','일','당일종가','전일종가','변동률','시가','고가','저가','거래량']]
bit_copy.head()

,날짜,당일종가,시가,고가,저가,거래량,변동률,연,월,일,종류
0,2022-12-26,16831.9,16831.8,16836.5,16822.0,133330.0,0.0000,2022,12,26,비트코인
1,2022-12-25,16831.8,16835.7,16852.9,16733.0,132890.0,-0.0003,2022,12,25,비트코인
2,2022-12-24,16837.2,16778.6,16855.8,16777.8,110120.0,0.0035,2022,12,24,비트코인
3,2022-12-23,16779.1,16820.5,16910.7,16768.0,184120.0,-0.0025,2022,12,23,비트코인
4,2022-12-22,16820.6,16831.8,16862.2,16566.1,198280.0,-0.0007,2022,12,22,비트코인
